In [1]:
import numpy as np
import numpy.polynomial.legendre as leg
import matplotlib.pyplot as plt
import scipy.special as sp
%matplotlib widget

# Obtain gauss-legendre quad points and eigenvalues
degree = 10
nodes, weights = leg.leggauss(degree)
evs = np.arange(degree) + 0.5
higher_degree = int(np.ceil(0.5 * (3.0 * degree - 1)))

higher_nodes, higher_weights = leg.leggauss(higher_degree)

In [2]:
# Build original basis Vandermonde matrix
van = np.array([[leg.legval(nodes[i], (*np.zeros(j), 1))
                 for j in range(degree)]
                for i in range(degree)])

inv_van = np.array([[weights[j] * evs[i] * leg.legval(nodes[j], (*np.zeros(i), 1))
                     for j in range(degree)]
                    for i in range(degree)])

In [17]:
# Function to test: elliptic cosine and its square
k = 0.99 # elliptic modulus
m = k ** 2.0 # elliptic parameter
quarter_period = sp.ellipk(m)

# Build grid: parameters
elements = 6
low, high = -quarter_period, quarter_period  # -np.pi, np.pi
length = high - low
dx = length / elements
J = dx / 2.0

# Build grid: construction
def build_grid(order):
    n_o, w_o = leg.leggauss(order)
    nodes_iso = (np.array(n_o) + 1) / 2
    # element left boundaries (including ghost elements)
    xl = np.linspace(low, high-dx, num=elements)
    # construct coordinates
    grid_o = np.zeros((elements, order))
    for i in range(elements):
        grid_o[i, :] = xl[i] + dx * np.array(nodes_iso)
    return grid_o

# Build grids
grid, higher_grid = build_grid(degree), build_grid(higher_degree)
cutoff = 20
fundamental = 2.0 * np.pi / length
wavenumbers = fundamental * np.arange(1-cutoff, cutoff)

# plt.figure()
# plt.plot(grid.flatten(), np.sin(grid).flatten(), 'o')
# plt.plot(higher_grid.flatten(), np.sin(higher_grid).flatten(), 'o')
# plt.grid(True), plt.tight_layout()

In [18]:
# Build quad matrix for Fourier integral of an interpolant on the quad grid
single_quad = np.multiply(weights[None, None, :], np.exp(-1j * np.tensordot(wavenumbers, grid, axes=0))) * J / length
print(single_quad.shape)

(39, 6, 10)


In [19]:
# Build quad matrix for Fourier integral of a product of interpolants on the quad grid
inner_kernel = np.array([[[higher_weights[s] * 
                           leg.legval(higher_nodes[s], (*np.zeros(p), 1)) *
                           leg.legval(higher_nodes[s], (*np.zeros(q), 1))
                           for s in range(higher_degree)]
                          for p in range(degree)]
                         for q in range(degree)])
modes_at_higher_degree = np.exp(-1j * np.tensordot(higher_grid, wavenumbers, axes=0))
double_quad_inner = np.transpose(np.tensordot(inner_kernel, modes_at_higher_degree, axes=([2], [1])), (2, 0, 1, 3))
# Do products with inverse vandermonde matrices
double_quad = np.tensordot(inv_van,
                           np.tensordot(inv_van,
                                        double_quad_inner,
                                        axes=([0], [1])),
                           axes=([0], [2]))
double_quad = np.transpose(double_quad, (3, 2, 0, 1)) * J / length
print(double_quad.shape)

(39, 6, 10, 10)


In [20]:
# Test function: sin^2(x)
f = sp.ellipj(grid.flatten(), m)[1].reshape(grid.shape)
f_sq = sp.ellipj(grid.flatten(), m)[1].reshape(grid.shape) ** 2.0

f_mult = np.einsum('mj,mk->mjk', f, f)

def cn_sq_spectrum(n):
    if n == 0:
        return 1.0 + (sp.ellipe(m) / sp.ellipk(m) - 1.0) / m
    # if (n+1) % 2 == 0:
    #     return 0
    else:
        K = sp.ellipk(m)
        Kp = sp.ellipk(1-m)
        return np.pi**2.0 / (2.0 * m * K ** 2.0) * n / np.sinh(n * np.pi * Kp/K)

spectrum_f = np.tensordot(single_quad, f_sq, axes=([1, 2], [0, 1]))
spectrum_f_double = np.tensordot(double_quad, f_mult, axes=([1, 2, 3], [0, 1, 2]))

# print(cn_sq_spectrum(-1))
print(cn_sq_spectrum(0))

waves = wavenumbers / fundamental

true_spectrum = np.array([cn_sq_spectrum(waves[i]) for i in range(waves.shape[0])])

plt.close('all')
fig, ax = plt.subplots(1, 2)
ax[0].plot(grid.flatten(), f_sq.flatten(), 'o--'), ax[0].grid(True)
ax[1].semilogy(waves, np.real(spectrum_f), 'o', label='Single-quad')  # , ax[1].plot(wavenumbers, np.imag(spectrum_f), 'o')
ax[1].semilogy(waves, np.real(spectrum_f_double), 'o', label='Double-quad')  # , ax[1].plot(wavenumbers, np.imag(spectrum_f_double), 'o')
ax[1].semilogy(waves, true_spectrum, 'o', label='True')
ax[1].legend(loc='best'), ax[1].grid(True)

error_single = np.divide(np.absolute(np.real(spectrum_f) - true_spectrum), tru
error_double = np.absolute(np.real(spectrum_f_double) - true_spectrum)

plt.figure()
plt.semilogy(waves, error_single, 'o', label='single error')
plt.semilogy(waves, error_double, 'o', label='double error')
plt.legend(loc='best'), plt.grid(True)

0.2923211863254005


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.legend.Legend at 0x23d5e6ade80>, None)